# Process subtitle text and extract part of speech considering pronoun resolution

In [1]:
# install spacy version 2.1.3 due to issues with the neural co ref library 
!pip install spacy==2.1.3


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 27.7 MB 1.6 MB/s 
     |████████████████████████████████| 82 kB 397 kB/s 
     |████████████████████████████████| 3.2 MB 37.4 MB/s 
     |████████████████████████████████| 184 kB 56.5 MB/s 
     |████████████████████████████████| 2.1 MB 46.3 MB/s 
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.3
    Uninstalling srsly-2.4.3:
      Successfully uninstalled srsly-2.4.3
  Attempting uninstall: preshed
    Found existing installation: preshed 3.0.6
    Uninstalling preshed-3.0.6:
      Successfully uninstalled preshed-3.0.6
  Attempting uninstall: blis
    Found existing installation: blis 0.7.7
    Uninstalling blis-0.7.7:
      Successfully uninstalled blis-0.7.7
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.17
    Uninstalling thinc-8.0.17:
      Successfully uninstalled thinc-8.0.17
  Attempting u

In [2]:
# install the neural co ref
!pip install neuralcoref

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 286 kB 4.2 MB/s 
     |████████████████████████████████| 132 kB 49.2 MB/s 
     |████████████████████████████████| 8.9 MB 49.4 MB/s 
     |████████████████████████████████| 79 kB 7.4 MB/s 
     |████████████████████████████████| 127 kB 63.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [5]:
# download the small english library
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.1 MB 4.3 MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.1.0-py3-none-any.whl size=11074433 sha256=3b1aac9fdf4c7948419255b6eb6bab2aee7dad4c18aa8362c4545959f2f18455
  Stored in directory: /tmp/pip-ephem-wheel-cache-ped71gix/wheels/59/4f/8c/0dbaab09a776d1fa3740e9465078bfd903cc22f3985382b496
Successfully built en-core-web-sm
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.3.0
    Uninstalling en-core-web-sm-3.3.0:
      Successfully uninstalled en-core-web-sm-3.3.0
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [8]:
# imports
import spacy
import neuralcoref

100%|██████████| 40155833/40155833 [00:04<00:00, 9664544.21B/s] 


In [9]:
#  load up the small model
nlp = spacy.load("en_core_web_sm")


In [10]:
# pop the neural co ref module into the pipeline

neuralcoref.add_to_pipe(nlp,greedyness=0.52)

# quick test to check it works
doc = nlp("Tell Sam that he will have to leave without Arthur, as he is sick.")
print(doc._.coref_resolved)

Tell Sam that Sam will have to leave without Arthur, as Sam is sick.


In [60]:
# load the sub title file
f = open("life_of_birds_epi_1_srt_text.txt")
doc = nlp(f.read())


In [61]:
# process the doc to replace the pronouns with the referenced nouns

resolved_doc = doc._.coref_resolved
print(resolved_doc[:500]) # first 500 characters

Birds are the most accomplished aeronauts the world has ever seen.
Birds fly high and low.
At great speed and very slowly.
And always with extraordinary precision and control  But birds are not the only creatures in the air.
There are also small furry mammals, bats, like birds in Texas.
birds are so competent in the air  that birds have just made a journey from Mexico, a thousand miles away,  simply in order to rear birds young in this cave,  which is particularly suitable for birds as a nursery


In [62]:
f_out = open("birds_resolved.txt", "w")
f_out.write(resolved_doc)
f_out.close()

In [63]:
# load the list of bird names extracted from wikipedia

bird_dict = {}
birds = open("bird_list.txt").read().lower().split("\n")
for bird in birds:
  # skip single letters A, B, C etc
  if len(bird) == 1:
    continue
  bird_dict[bird] = True
  # insert the last word too e..g yellow crested parrot > parrot
  bird_dict[bird.split(" ")[-1]] = True

# check it has worked
list(bird_dict.keys())[:5]

["abbott's babbler", 'babbler', "abbott's booby", 'booby', "abbott's starling"]

## Extract the nouns adj and proper nouns highlighting bird names

In [64]:
# split the subtitles into a set of rows for processing
resolved_subtitles = open("birds_resolved.txt").read()

subtitle_rows = resolved_subtitles.split("\n")
print(subtitle_rows[:5])


['Birds are the most accomplished aeronauts the world has ever seen.', 'Birds fly high and low.', 'At great speed and very slowly.', 'And always with extraordinary precision and control  But birds are not the only creatures in the air.', 'There are also small furry mammals, bats, like birds in Texas.']


In [73]:
# dump the filtered resolved subtitles to a file

valid_tokens = ["NOUN", 'ADJ', 'PROPN']

filtered_subtitle_file = open("birds_resolved_filtered.txt", "w")
for row in subtitle_rows:
  filtered_subtitle_file.write(row + "\n")
  row_doc = nlp(row)
  for token in row_doc:
    if token.pos_ not in valid_tokens:
      continue
    is_bird = token.pos_ == "NOUN" and token.lemma_ in bird_dict
    filtered_subtitle_file.write(token.text + "\t" + token.lemma_ + "\t" + token.pos_ + "\t" + str(is_bird) + "\n")
  filtered_subtitle_file.write("\n")


filtered_subtitle_file.close()